In [ ]:
import os, sys
import numpy as np

bayes_net = [('D', 0.2), ('I', 0.1), ('G|ID', [[0.1, 0.2], [0.2, 0.5]]), ('S|I', [0.3, 0.2]), ('R|G', [0.6, 0.2])]

def make_tensors(formula):
    """ Preprocess probability distributions into form of tensors (axes, 2^len(axes)-array),
          containing the complete distribution with negative cases. """
    return [(axes[0]+axes[2:] if '|' in axes else axes, np.array([np.reshape(1.0 - dist.ravel(), dist.shape), dist])) 
        for axes, dist in ((_, np.array(_dist)) for _, _dist in formula) ]

# Variable Elimination

In [2]:
def ve(order, formula, **query):
    """ Variable Elimination algorithm that computes marginal probabilities for a Bayesian network """
    tensors = make_tensors(formula)
    while order:
        var, order = order[0], order[1:]
        # Leave in the summation for Xi only factors mentioning Xi
        factors = list(filter(lambda i: var in i[0], tensors))
        # Multiply the factors, getting a factor that contains a number for each value of the variables mentioned, including Xi
        axes, tensor_out = ve_product(factors)
        # Sum out Xi, getting a factor f that contains a number for each value of the variables mentioned, not including Xi
        tensor_out = np.sum(tensor_out, axis = axes.index(var))
        axes.remove(var)
        # Replace the multiplied factor in the summation
        tensors = list(filter(lambda i: var not in i[0], tensors))
        tensors.append((''.join(axes), tensor_out))
    if query:
        result_axes, result_tensor = ve_product(tensors)
        return result_tensor[tuple(map(query.get, result_axes))]
    else: # Default query asks for probability of the positive case for the only remaining variable
        return tensors[0][1][1]

def ve_product(factors):
    """ Tensor point-wise multiplication """
    axes = list(set(''.join(map(lambda i: i[0], factors))))
    tensor_out = np.zeros((2,) * len(axes))
    for assignments in range(1<<len(axes)):
        # Maps axes of tensor to a given set of index assignments
        idx = lambda a: tuple(map({v:((assignments>>(len(axes)-1-i))&1) for i, v in enumerate(axes)}.get, a))
        tensor_out[idx(axes)] = np.prod([i[1][idx(i[0])] for i in factors])
    return axes, tensor_out

In [3]:
ve('IDGR', bayes_net)

0.29000000000000004

In [4]:
ve('IDSR', bayes_net)

0.13400000000000001

In [5]:
ve('SR', bayes_net, G=1, I=0, D=0) / ve('GSR', bayes_net, I=0, D=0)

0.10000000000000001

In [6]:
ve('DGR', bayes_net, S=1, I=1) / ve('IDGR', bayes_net)

0.068965517241379309

In [7]:
ve('DGS', bayes_net, R=1, I=1) / ve('IDGS', bayes_net)

0.09077598828696927

# Direct Sampling

In [ ]:
def ds(order, formula, **query):
    """ Direct Sampling algorithm that computes marginal probabilities for a Bayesian network
        `order` should be a topological order of the casual DAG. """
    tensors = make_tensors(formula)